In [1]:
import matplotlib.pyplot as mpl
from analyze import *
from utils import *
from TextBag import TextBag as GraphGenerator
from multiprocessing import Pool

N_CORES = 8
emb, word2id_dict, id2word_dict  = load_embeddings()

Embeddings loaded


In [2]:
print(gen_style_vec)

<function gen_style_vec at 0x7f212d94b2f0>


In [10]:
def vectorize(path):
    bag = GraphGenerator(path, emb, word2id_dict, id2word_dict)
    G = bag.generate_graph()
    if nx.number_of_nodes(G) <= 400:
        return gen_style_vec(G)
    else:
        return None

def main():    
    presidents = os.listdir('./data/presidential/')
    p = Pool(N_CORES)
    pres_speeches = []
    for president in presidents:
        text_files = os.listdir('./data/presidential/' + president)
        for speech in text_files:
            path = './data/presidential/' + president + '/' + speech
            pres_speeches += [(president, path)]
    speeches = [x[1] for x in pres_speeches]
    return p.map(vectorize, speeches), pres_speeches

finished, pres_speeches = main()

962
('mckinley', './data/presidential/mckinley/mckinley_speeches_003.txt')


TypeError: 'NoneType' object is not iterable

In [4]:
vectors = [(pres, file, finished[i]) for i, (pres, file) in enumerate(pres_speeches) if finished[i] is not None]

In [5]:
pres = [x[0] for x in vectors]
X = np.concatenate([x[2][:,np.newaxis] for x in vectors], axis=1)

In [6]:
from sklearn import manifold
from sklearn.decomposition import PCA

pca = PCA(10)
X_r = pca.fit(X[19:,:].T).transform(X[19:,:].T)
tsne = manifold.TSNE()
X_embedded = tsne.fit_transform(X_r)
print(X_embedded.shape)

(214, 2)


In [7]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools

plotly.tools.set_credentials_file(username='amanjuna', api_key='v5RvlMiKGJwt6X5yxmUQ')

plots = []
for president in set(pres):
    occur = [i for i, x in enumerate(pres) if x == president]
    plots.append(go.Scatter(x = X_embedded[occur,0], y = X_embedded[occur,1], mode='markers', name=president))
fig = go.Figure(data=plots)
py.iplot(fig, filename="whatever")

/home/amanjuna/.local/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead

